In [89]:
import pandas as pd
import sys, os


In [90]:
# task 2: load census data

data = pd.read_csv('../data/acs2015_census_tract_data.csv') # the original
df = data # clone for funsies
df.drop(df[df['State'] == 'Puerto Rico'].index, inplace=True)

df.head(2)

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3


In [91]:
# task 3: Find state with highest population density based on race

# example: white

# create a column of totalpop*race%

df['ActualPop'] = (df['Hispanic'] / 100) * df['TotalPop']
df.head(5)

# aggregate by state

#df['ActualPop'].group
# sum of white column per state
#perState = df.groupby('State')['Hispanic'].sum().idxmax()
#perStateNum = df.groupby('State')['Hispanic'].sum().max()

#perStateNum

#bargain_wine = df.loc[(df.points/df.price).idxmax, 'title'] #idxmax gets max index/id

# cast to int at the end

# return a tuple: state, max_population



,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,ActualPop
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4,17.532
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3,17.248
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2,0.000
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8,464.415
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2,75.341


In [92]:
def get_max_state_population_by_race(data, race):
    # Calculate the state with the maximum population for a specified race
    
    # copy dataframe
    df = data

    # create a new column with actual number of people by race
    # multiply (data[race]/100) * data['TotalPop']
    df['ActualPop'] = (df[race] / 100) * df['TotalPop']

    # aggregate by state
    perState = df.groupby('State')[race].sum().idxmax()
    perStateNum = df.groupby('State')[race].sum().max()

    

    #return state, population_value
    #return df['State'], df['TotalPop']
    return perState, perStateNum

print(get_max_state_population_by_race(df, 'Hispanic'))

('California', 298782.5)


In [93]:
def get_unemployment_rate(data, rate):
    
    # make a new column TotalPop * Unemployment/100

    df = data
    df['UnemployedPop'] = df['TotalPop'] * df['Unemployment']/100

    # aggregate by state

    stateRate = 0
    aggRate = 0

    # for 'highest' get idxmax()
    if rate == 'highest':
        stateRate = df.groupby('State')['UnemployedPop'].sum().idxmax()
        aggRate = df.groupby('State')['UnemployedPop'].sum().max()

    # for 'lowest' get idxmin()
    elif rate == 'lowest':
        stateRate = df.groupby('State')['UnemployedPop'].sum().idxmin()
        aggRate = df.groupby('State')['UnemployedPop'].sum().min()
    
    # return tuple: state, rate
    return stateRate, aggRate

df = data
df['UnemployedPop'] = df['TotalPop'] * df['Unemployment']/100

# aggregate by state

# for 'highest' get idxmax()
highestState = df.groupby('State')['UnemployedPop'].sum().idxmax()
highestRate = df.groupby('State')['UnemployedPop'].sum().max()

# for 'lowest' get idxmin()
lowestState = df.groupby('State')['UnemployedPop'].sum().idxmin()
lowestRate = df.groupby('State')['UnemployedPop'].sum().min()

print(f"Lowest: {lowestState}, {lowestRate} \nHighest: {highestState}, {highestRate}")


Lowest: North Dakota, 21269.568 
Highest: California, 3898336.959


## Task 5: Get Income Inequalities

In [113]:
def get_income_inequality(data):
    # copy 
    df = data

    # filter records based Income and Poverty
    #df['Income'] > 50000 and df['Poverty'] > 50%

    df = df.loc[df.Income >= 50000]
    
    df = df.loc[df.Poverty >= 5]

    #df.head(5)

    # drop unnecessary rows
    df = df.drop(columns=['TotalPop', 'Men', 'Women', 'Citizen', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment'])

    # return dataFrame, with correct filter_records and columns
    return df
    

    # (the test checks that the datatype is right)


    return df


get_income_inequality(df)
#df.loc[(df.Income >= 50000) & (df.Poverty >= 5)]
#df.loc[df.Poverty >= 5]

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,ActualPop,UnemployedPop
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4,17.532,105.192
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2,75.341,452.046
5,1001020600,Alabama,Autauga,3851,1787,2064,13.1,72.9,11.9,0.0,...,4.5,19.8,1560,79.4,14.7,5.8,0.0,10.9,504.481,419.759
7,1001020801,Alabama,Autauga,3187,1502,1685,1.3,84.0,10.7,3.1,...,5.9,24.3,1502,78.1,14.8,7.1,0.0,8.2,41.431,261.334
8,1001020802,Alabama,Autauga,10915,5486,5429,1.4,89.5,8.4,0.0,...,1.3,29.4,4348,73.3,22.1,4.6,0.0,8.7,152.810,949.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73048,56041975200,Wyoming,Uinta,6629,3395,3234,3.7,94.0,0.2,0.0,...,3.5,28.0,3288,72.6,21.8,5.6,0.0,3.7,245.273,245.273
73049,56041975300,Wyoming,Uinta,7220,3437,3783,6.3,87.7,0.3,1.0,...,1.6,17.7,3484,76.6,19.8,3.6,0.0,2.7,454.860,194.940
73051,56043000200,Wyoming,Washakie,3256,1615,1641,12.3,84.7,1.8,0.3,...,9.2,15.3,1529,70.9,21.9,6.0,1.2,4.2,400.488,136.752
73053,56043000302,Wyoming,Washakie,2566,1265,1301,18.6,76.2,0.0,0.5,...,2.0,12.6,1188,72.9,19.4,7.7,0.0,10.1,477.276,259.166


## Task 6: Get states that are mostly feminine

In [ ]:
def get_total_feminine_population(data):
    
    # copy
    df = data
    # filter records based Population and Women
    # total pop <= 10,000
    df = df.loc[df.TotalPop > 10000]
    # % women <= 57%
    df = df.loc[df.Women/df.TotalPop > 0.57]

    # return ddataframe with correct filter_records and columns
    return df



## Task 7: Get Diversity Information

In [116]:
def get_most_diverse_racially(data):
    
    # copy
    df = data

    # creating new column where 4 or more races = 15

    # make a point system where if count >= 4 then in new_col

    count = 0

    count = 0

    if df['Hispanic'] >= 15:
        count += 1
    if df['White'] >= 15:
        count += 1
    if df['Black'] >= 15:
        count += 1
    if df['Native'] >= 15:
        count += 1
    if df['Asian'] >= 15:
        count += 1
    if df['Pacific'] >= 15:
        count += 1

    #df['DiversityInd'] = [x if x ]
    # df_data.drop(df_data[df_data['adults'] == 0].index, inplace=True)    

        
    # filter records that have at least 4 of 6 races with 15% or more
    

    # return dataframe with correct filter_records and columns

    pass

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().